In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## 1. Importation des bibliothèques

- **`numpy`** est utilisé pour les opérations numériques.
- **`pandas`** est utilisé pour la manipulation des données.
- **`RandomForestClassifier`** et **`LogisticRegression`** sont des algorithmes de machine learning importés depuis `sklearn` pour entraîner des modèles.
- **`train_test_split`** est une fonction pour diviser les données en ensembles d'entraînement et de test.


In [2]:
import numpy as np
import pandas as pd

#The Machine learning alogorithm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Test train split
from sklearn.model_selection import train_test_split

## 2. Changement de répertoire de travail

- **`os`** est utilisé pour interagir avec le système d'exploitation.
- **`os.chdir`** change le répertoire de travail courant vers le répertoire spécifié, ici pour Kaggle.


In [3]:
import os
os.chdir("/kaggle/working/") 

## 3. Chargement et exploration des données

- **`pd.read_csv`** lit le fichier CSV des données d'entraînement et le charge dans un DataFrame.
- **`train.train.head()`** affiche les premières lignes du DataFrame pour une vue d'ensemble rapide des données.

In [4]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 4. Préparation des données

- Sélectionnez les colonnes **"Pclass"** et **"Sex"** pour créer un DataFrame **`train_x`** qui sera utilisé comme ensemble de caractéristiques pour entraîner les modèles.
- **`train_x.head()`** affiche les premières lignes de ces données.


In [7]:
train_entrainement = train[["Pclass", "Sex"]]
train_entrainement.head()

,Pclass,Sex
0,3,male
1,1,female
2,3,female
3,1,female
4,3,male


- Sélectionnez la colonne **"Survived"** pour créer un DataFrame **`train_cible`** qui sera utilisé comme cible pour entraîner les modèles.
- **`train_cible.head()`** affiche les premières lignes de cette cible.

In [8]:
train_cible = train[["Survived"]]
train_cible.head()

,Survived
0,0
1,1
2,1
3,1
4,0


## 5. Division des données en ensembles d'entraînement et de validation

- Divisez les données en ensembles d'entraînement (**tr_x**, **tr_y**) et de validation (**cv_x**, **cv_y**).
- L'argument **`test_size=0.30`** indique que 30% des données seront utilisées pour la validation.
- Remplacez les valeurs de la colonne **"Sex"** par des valeurs numériques (1 pour "male" et 0 pour "female") en utilisant la méthode **`replace`**.
- Cette transformation rend la colonne compatible avec les algorithmes de machine learning.

In [9]:
train_entrainement.loc[:, "Sex"] = train_entrainement["Sex"].replace({"male": 1, "female": 0})
train_entrainement.head()

/tmp/ipykernel_17/1058115740.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_entrainement.loc[:, "Sex"] = train_entrainement["Sex"].replace({"male": 1, "female": 0})


,Pclass,Sex
0,3,1
1,1,0
2,3,0
3,1,0
4,3,1


## 5. Division des données en ensembles d'entraînement et de validation

- Divisez les données en ensembles d'entraînement (**tr_entrainement**, **tr_cible**) et de validation (**cv_entrainement**, **cv_cible**).
- L'argument **`test_size=0.30`** indique que 30% des données seront utilisées pour la validation.


In [10]:
tr_entrainement, cv_entrainement, tr_cible, cv_cible   = train_test_split(train_entrainement, train_cible, test_size = 0.30)

print(tr_entrainement.head())
print(tr_cible.head())

     Pclass Sex
620       3   1
736       3   0
335       3   1
768       3   1
363       3   1
     Survived
620         0
736         0
335         0
768         0
363         0


In [11]:
tr_cible = tr_cible.values.ravel()

In [12]:
rf = RandomForestClassifier()

In [13]:
rf.fit(tr_entrainement, tr_cible)
tr_entrainement.head()

,Pclass,Sex
620,3,1
736,3,0
335,3,1
768,3,1
363,3,1


In [14]:
accuracy = rf.score(cv_entrainement, cv_cible)
print("Accuracy = {}%".format(accuracy * 100))

Accuracy = 76.86567164179104%


In [15]:
lgr = LogisticRegression()

In [16]:
lgr.fit(tr_entrainement, tr_cible)

LogisticRegression()

In [17]:
accuracy = lgr.score(cv_entrainement, cv_cible)
print("Accuracy = {}%".format(accuracy * 100))


Accuracy = 77.98507462686567%


In [18]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
test_entrainement = test[["Pclass", "Sex"]]
test_entrainement.head()

,Pclass,Sex
0,3,male
1,3,female
2,2,male
3,3,male
4,3,female


In [20]:
test_entrainement.loc[:, "Sex"] = train_entrainement["Sex"].replace({"male": 1, "female": 0})
test_entrainement.head()

/tmp/ipykernel_17/1531927447.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_entrainement.loc[:, "Sex"] = train_entrainement["Sex"].replace({"male": 1, "female": 0})


,Pclass,Sex
0,3,1
1,3,0
2,2,0
3,3,0
4,3,1


In [21]:
test_entrainement.head()

,Pclass,Sex
0,3,1
1,3,0
2,2,0
3,3,0
4,3,1


In [22]:
prd = rf.predict(test_entrainement)

In [23]:
prd

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [24]:
op = test[['PassengerId']].copy()

In [25]:
op['Survived'] = prd

In [26]:
op.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,0


In [27]:
op.to_csv("Submission.csv",index=False)